ML Course, Bogotá, Colombia  (&copy; Josh Bloom; June 2019)

In [68]:
%run ../talktools.py

## Classification with Keras

We just saw how to do regression problems with neural nets in `keras`. Let's now explore classification. Because we'll use this dataset later, let's introduce the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist#labels) dataset: 70k small (28$\times$28) images of 10 different types of clothing.

<img src="https://github.com/zalandoresearch/fashion-mnist/blob/master/doc/img/fashion-mnist-sprite.png?raw=true" width="80%">

Each training and test example is assigned to one of the following labels:

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

Tensorflow has a simple method to get this data locally

In [ ]:
import datetime, os

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import tensorflow as tf

# Print keras version
print(tensorflow.keras.__version__)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # scale the images to 0-1

In [ ]:
x_train.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ind = 1
plt.axis('off')
plt.imshow(x_train[ind], cmap=plt.cm.gray_r, interpolation='nearest')

In [ ]:
y_train[ind]

For now, let's treat every pixel as a separate input and build a keras NN to predict the output.

In [ ]:
input_shape = x_train[0].shape
input_shape

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(512, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [71]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
# define path to save model
model_path = f'nn_results/colombia_nn_{run_time_string}.h5'
print(f"Training ... {model_path}")

logdir = os.path.join("nn_results", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.75,
                              patience=2, min_lr=1e-6, verbose=1, cooldown=0)

csv_logger = tf.keras.callbacks.CSVLogger(f'nn_results/training_{run_time_string}.log')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, 
                                             patience=3, \
                                             verbose=1, mode='auto')

model_check = tf.keras.callbacks.ModelCheckpoint(model_path,
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=20, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback, reduce_lr, csv_logger, earlystop, model_check])

Training ... nn_results/colombia_nn_2019-05-21T16:49.h5
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
59712/60000 [============================>.] - ETA: 0s - loss: 0.2266 - accuracy: 0.9158
Epoch 00001: val_accuracy improved from -inf to 0.88210, saving model to nn_results/colombia_nn_2019-05-21T16:49.h5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.2266 - accuracy: 0.9158 - val_loss: 0.3481 - val_accuracy: 0.8821
Epoch 2/20
59520/60000 [============================>.] - ETA: 0s - loss: 0.2215 - accuracy: 0.9174
Epoch 00002: val_accuracy improved from 0.88210 to 0.88270, saving model to nn_results/colombia_nn_2019-05-21T16:49.h5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.2216 - accuracy: 0.9173 - val_loss: 0.3594 - val_accuracy: 0.8827
Epoch 3/20
59520/60000 [============================>.] - ETA: 0s - loss: 0.2161 - accuracy: 0.9200
Epoch 00003: val_accuracy did not improve from 0.88270
60000/60000 [==============

You'll notice above that the accuracy is much higher than the val_accuracy. That is, we overfit on the training data. One way to help protect against this is to introduce `Dropout`

<img src="https://cdn-images-1.medium.com/max/1600/1*iWQzxhVlvadk6VAJjsgXgg.png">

Srivastava, Nitish, et al. ”Dropout: a simple way to prevent neural networks from
overfitting”, JMLR 2014

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(512, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))  # 20% chance of dropping a node during training
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
# define path to save model
model_path = f'nn_results/colombia_nn_{run_time_string}.h5'
print(f"Training ... {model_path}")

logdir = os.path.join("nn_results", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.75,
                              patience=2, min_lr=1e-6, verbose=1, cooldown=0)

csv_logger = tf.keras.callbacks.CSVLogger(f'nn_results/training_{run_time_string}.log')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, 
                                             patience=3, \
                                             verbose=1, mode='auto')

model_check = tf.keras.callbacks.ModelCheckpoint(model_path,
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=20, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback, reduce_lr, csv_logger, earlystop, model_check])

Let's make some predictions

In [ ]:
model.predict(x_test)

In [ ]:
y_pred = model.predict_classes(x_test)
y_pred

In [ ]:
y_test

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)

import seaborn as sns
sns.set_context("poster")
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
sns.heatmap(conf_mat_normalized)
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
conf_mat

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [ ]:
lookup = {0: "T-shirt/top",
          1: "Trouser",
          2: "Pullover",
          3: "Dress",
          4: "Coat",
          5: "Sandal",
          6: "Shirt",
          7: "Sneaker",
          8: "Bag",
          9: "Ankle boot"}

In [ ]:
ind_wrong = []
for i, (pred, actual) in enumerate(zip(model.predict_classes(x_test),y_test)):
    if pred != actual:
        ind_wrong.append((i,pred, actual))

In [ ]:
ind = 100
plt.imshow(x_test[ind_wrong[ind][0]], cmap=plt.cm.gray_r, interpolation='nearest')
plt.axis("off")
plt.title(f"pred={lookup[ind_wrong[ind][1]]} true={lookup[ind_wrong[ind][2]]}")

How'd we do? The creators of the dataset maintain a website with results using `sklearn`:

http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/#